# Redes Sociais - APS 2

### Alunos: Arthur Barreto, Enricco Gemha e Felipe Catapano


Uma rede de artistas do Spotify que se conectam através de participar em determinada música, que foi topo das paradas em alguma semana no mundo, durante o período de 28/09/2013 a 09/10/2022. Os vértices representam artistas do Spotify e uma aresta não-direcionada indica uma música feita em parceria por dois artistas.

_Nota: os grafos são não-dirigidos, conforme informado no agregador de databases [Kaggle](https://www.kaggle.com/datasets/jfreyberg/spotify-artist-feature-collaboration-network), do qual foi extraído a base utilizada aqui._

## Pré-requisitos

In [1]:
import graph_tool_extras as gte
import netpixi
from graph_tool import draw
import distribution as dst

In [2]:
PATH = 'edges.csv'
PATH_NOS = 'nodes.csv'

## Análise dos dados importados

O arquivo que informa sobre os nos é o arquivo `nodes.csv`. Segue a analise inicial da quantidade de nos:

In [3]:
with open(PATH_NOS) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')
        print(parts[0])

        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 5:
            break

48WvrUGoijadXXCsGocwM4
4lDiJcOJ2GLCK6p9q5BgfK
652XIvIBNGg3C0KIGEJWit
3dXC1YPbnQPsfHPVkm1ipj
74terC9ol9zMo8rfzhSOiG


### Extratificando a amostra no número de nos

In [4]:
dict_nos = {}

In [5]:
with open(PATH_NOS) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    qtd_nos = 1
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        no = parts[0]

        if no not in dict_nos:
            dict_nos[no] = qtd_nos
            qtd_nos += 1
            
        i += 1
    

In [6]:
print(f'há {qtd_nos} nos')

há 156321 nos


Visualizando os valores do dicionário:

In [7]:
list(dict_nos.keys())[:5]

['48WvrUGoijadXXCsGocwM4',
 '4lDiJcOJ2GLCK6p9q5BgfK',
 '652XIvIBNGg3C0KIGEJWit',
 '3dXC1YPbnQPsfHPVkm1ipj',
 '74terC9ol9zMo8rfzhSOiG']

In [8]:
list(dict_nos.values())[:5]

[1, 2, 3, 4, 5]

A abordagem serrá sortear indices, que variam de 1 até `qtd_nos`. Após isso, basta verificar os indices sorteados que estão na lista de valores do dicionário criando.

In [9]:
import numpy as np
from numpy.random import choice

lisIdx = np.arange(1,qtd_nos + 1, 1)
lisProb = np.ones(shape=len(lisIdx), dtype=lisIdx.dtype)
lisProb = np.divide(lisProb, len(lisProb))
idxSort = choice(lisIdx, int(75e3), p = lisProb, replace = False)
idxSort.sort()
idxSort[:5]

array([ 3,  4,  7, 11, 12])

Para facilitar a verificação dos nos considerados, basta inverter o dionário, desta forma, as chaves serão os indices e os valores os nos em si.

In [10]:
idx_no = {v: k for k, v in dict_nos.items()}

In [11]:
list(idx_no.keys())[:5]

[1, 2, 3, 4, 5]

In [12]:
noStrat = {idx_no[idx]: idx for idx in idxSort}

In [13]:
list(noStrat.values())[:5]

[3, 4, 7, 11, 12]

Neste ponto, temos quais indices do dicionário `idx_no` a acessar, para então associar uma aresta.

O arquivo relevante para a próxima análise é `edges.csv`, que representa uma lista de valores separados por _vírgulas_, com uma aresta por linha. O significado de cada coluna do arquivo é:

- Primeira coluna: ID do nó de participação em música
- Segunda coluna: ID do nó de participação em música

Agora, devemos inspecionar os valores armazenados em `edges.csv`:

Como visto nas saidas acima, temos 2 arestas possíveis dentro os nos validos para as 10 primerias arestas válidas. 

## Criação do grafo

Utilizaremos a biblioteca [graph-tool](https://graph-tool.skewed.de/) somente para criação e visualização básica dos grafos, sem suporte de nenhum método ou função que não seja essencial.

In [15]:
g = gte.Graph(directed=False) # pois o grafo não é direcionado, como informado acima.

Antes, vamos definir duas funções auxiliares para facilitar a adição de novos nós e arestas, respectivamente.

In [16]:
def get_or_add_vertex(g, id):
    u = g.vertex_by_id(id)
    if u is None:
        u = g.add_vertex_by_id(id)
    return u

def get_or_add_edge(g, id1, id2):
    e = g.edge_by_ids(id1, id2)
    if e is None:
        e = g.add_edge_by_ids(id1, id2)
    return e

Depois de criar o novo grafo, vamos armazenar os valores de `edges.csv` nele, o transformando em uma rede.

In [14]:
with open(PATH) as file:

    # cria index de contagem para o loop abaixo.
    i = 0
    
    # ignora o cabeçalho.
    next(file)

    # Para não sobrecarregar este notebook
    # vamos espiar somente as 5 primeiras linhas.
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')
        no1 = parts[0]
        no2 = parts[1].split('\n')[0]

        # Imprime o nó A que referencia e o nó B que é referenciado.
        
        if (no1 and no2) in noStrat:
            print(f'bateu em i={i}')
        
        # Para não sobrecarregar este notebook, vamos usar um contador
        # e um break para imprimir apenas as cinco primeiras linhas.
        i += 1
        if i == 10:
            break

bateu em i=2
bateu em i=4
bateu em i=5
bateu em i=7
bateu em i=9


In [17]:
with open(PATH) as file:

    # Cria index de contagem de linhas lidas.
    i = 1
    j = 0
    
    # Ignora o cabeçalho.
    next(file)

    # Itera linha a linha do arquivo `out.linux`
    for line in file:

        # Transforma a linha em uma lista de partes,
        # considerando a vírgula como separador.
        parts = line.split(',')

        # Define os IDs de origem e destino.
        no1 = parts[0]
        no2 = parts[1].split('\n')[0]

        if (no1 and no2) in noStrat:
            # Adiciona os vértices.
            get_or_add_vertex(g, no1)
            get_or_add_vertex(g, no2)
    
            # Adiciona a aresta correspondente a esta linha.
            get_or_add_edge(g, no1, no2)

            j += 1
        
        # Incrementa o contador de linhas lidas.
        i += 1

# Imprime a quantidade de linhas lidas.
print(f'Foram lidas {i} linhas.') 
print(f'Foram considerados {j} arestas')

Foram lidas 300387 linhas.
Foram considerados 141911 arestas


A seguir, devemos chamar `draw.sfdp_layout`, passando a rede, para rodar um algoritmo de posicionamento chamado SFDP [[1](#sfdp)].

Esse algoritmo usa uma ideia conhecida como [force-directed graph drawing](https://en.wikipedia.org/wiki/Force-directed_graph_drawing) para posicionar os vértices de forma a evidenciar agrupamentos.

In [ ]:
layout = draw.sfdp_layout(g)

In [ ]:
gte.move(g, layout)

## Armazenamento da rede

Para garantir a segurança da informação processada, devemos guardá-la em um arquivo na mesma pasta deste notebook.

In [ ]:
g = gte.clean(g)
gte.save(g, 'spotify.net.gz')

## Visualização da rede

O próximo passo é a renderização da rede.

In [ ]:
r = netpixi.render('spotify.net.gz', infinite=True)

Por fim, devemos ajustar a visualização da renderização.

In [ ]:
r.vertex_default(size=4, bwidth=1)

In [ ]:
r.edge_default(width=1)

## Estatísticas

In [ ]:
print(f"Número de vértices: {len(g.get_vertices())}")
print(f"Número de arestas: {len(g.get_edges())}")

In [ ]:
print(f"Densidade da rede: {g.density()}")
print(f"Transitividade da rede: {g.transitivity()}")

In [ ]:
degrees = g.get_total_degrees()

In [ ]:
degrees.describe()

In [ ]:
degrees.hist(bins = int(np.sqrt(len(g.get_edges() ) ) ) );

In [ ]:
dst.not_normal(degrees)

In [ ]:
dst.more_powerlaw_than_lognormal(degrees)

In [ ]:
dst.more_powerlaw_than_exponential(degrees)

In [ ]:
distances = g.get_distances()

In [ ]:
distances.describe()

In [ ]:
distances.hist();